In [1]:
from database import init_db, db_session
from bankmodel import Customer
from sqlalchemy import select


init_db()

In [2]:
# ORM객체 쿼리
customers = Customer.query.all()
customers

[<1c, 원칠4, 2, 500, silver>,
 <2c, 이서혁, 1, 2000, gold>,
 <3c, aiot, 3, 50000, vip>,
 <4c, 새싹, 4, 200000, vvip>]

In [5]:
c = db_session.get(Customer, '1c')
c

<1c, 최원칠, 2, 500, silver>

In [6]:
Customer.query.filter(Customer.c_id =='1c')[0]

<1c, 최원칠, 2, 500, silver>

In [7]:
# db_session.execute()로 SQL쿼리 가능
# 튜플로 가져오므로 스칼라() 처리 필요
q = db_session.execute(select(Customer)).scalars()
for c in q:
    print(c)

<1c, 최원칠, 2, 500, silver>
<2c, 이서혁, 1, 2000, gold>
<3c, aiot, 3, 50000, vip>
<4c, 새싹, 4, 200000, vvip>


In [9]:
# 등급 순차정렬인데, 문자열 기준으로 정렬 된듯
q = db_session.execute(select(Customer).order_by(Customer.rat)).scalars()
for c in q:
    print(c)

<2c, 이서혁, 1, 2000, gold>
<1c, 최원칠, 2, 500, silver>
<3c, aiot, 3, 50000, vip>
<4c, 새싹, 4, 200000, vvip>


In [10]:
q = db_session.execute(select(Customer).where(Customer.name =='aiot')).scalars()
for c in q:
    print(c)

<3c, aiot, 3, 50000, vip>


In [12]:
# 데이터 수정
customers[0].name = '원칠3'
db_session.commit() # 커밋 필수
customers = Customer.query.all()
customers

[<1c, 원칠3, 2, 500, silver>,
 <2c, 이서혁, 1, 2000, gold>,
 <3c, aiot, 3, 50000, vip>,
 <4c, 새싹, 4, 200000, vvip>]

In [13]:
customers[0].name = '원칠4' # 객체에 직접 입력시 add 생략
customers[0].verified = True
db_session.commit()
customers = Customer.query.all()
customers

[<1c, 원칠4, 2, 500, silver>,
 <2c, 이서혁, 1, 2000, gold>,
 <3c, aiot, 3, 50000, vip>,
 <4c, 새싹, 4, 200000, vvip>]

In [10]:

c = Customer(c_id = '12c', name = '데이터베스')
db_session.add(c) 
db_session.commit()
customers = Customer.query.all()
customers

C:\Users\SBAUser\AppData\Local\Temp\ipykernel_12392\890457631.py:3: SAWarning: New instance <Customer at 0x2ac6cf54220> with identity key (<class 'bankmodel.Customer'>, ('12c',), None) conflicts with persistent instance <Customer at 0x2ac6cf55540>
  db_session.commit()
C:\Users\SBAUser\AppData\Local\Temp\ipykernel_12392\890457631.py:3: SAWarning: Identity map already had an identity for (<class 'bankmodel.Customer'>, ('12c',), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  db_session.commit()


[<1c, 원칠4, 2, 500, silver>,
 <2c, 이서혁, 1, 2000, gold>,
 <3c, aiot, 3, 50000, vip>,
 <4c, 새싹, 4, 200000, vvip>,
 <12c, 데이터베스, 0, 0, normal>]

In [14]:
from bankmodel import Accounts
# accounts 테이블의 모든 계좌를 쿼리후 출력
accounts = Accounts.query.all()
accounts

[<11a, 100, 1c>,
 <12a, 400, 1c>,
 <21a, 2000, 2c>,
 <31a, 10000, 3c>,
 <32a, 20000, 3c>,
 <33a, 20000, 3c>,
 <41a, 100000, 4c>,
 <42a, 50000, 4c>,
 <43a, 20000, 4c>,
 <44a, 30000, 4c>]

In [15]:
# get함수를 활용해 a_id가 11a 인 계좌 출력
a = db_session.get(Accounts, '11a')
a

<11a, 100, 1c>

In [16]:
# 쿼리 filter 함수를 활용해 c_id가 3c인 계좌 출력
Accounts.query.filter(Accounts.c_id =='3c').all()

[<31a, 10000, 3c>, <32a, 20000, 3c>, <33a, 20000, 3c>]

In [17]:
# a_id가 45a이고 c_id가 4c인 계좌 생성후 DB에 추가
# a_id = '45a'
# if db_session.get(Accounts, a_id) is None:
a = Accounts(a_id = '45a', c_id = '4c')
db_session.add(a)
db_session.commit()
accounts = Accounts.query.all()
accounts

[<11a, 100, 1c>,
 <12a, 400, 1c>,
 <21a, 2000, 2c>,
 <31a, 10000, 3c>,
 <32a, 20000, 3c>,
 <33a, 20000, 3c>,
 <41a, 100000, 4c>,
 <42a, 50000, 4c>,
 <43a, 20000, 4c>,
 <44a, 30000, 4c>,
 <45a, 0, 4c>]

In [20]:
# a_id가 21a인 계좌를 가져온후 금액을 4000으로 바꾸고 DB업데이트
# a = Accounts.query.filter(Accounts.a_id =='21a')[0]
a = db_session.get(Accounts, '21a')
a.amount = 4000
a.verified = True
db_session.commit()
accounts = Accounts.query.all()
accounts

[<11a, 100, 1c>,
 <12a, 400, 1c>,
 <21a, 4000, 2c>,
 <31a, 10000, 3c>,
 <32a, 20000, 3c>,
 <33a, 20000, 3c>,
 <41a, 100000, 4c>,
 <42a, 50000, 4c>,
 <43a, 20000, 4c>,
 <44a, 30000, 4c>,
 <45a, 0, 4c>]